The bank marketing dataset was first presented in Moro, S., Laureano, R. and Cortex, P. (2011). The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. In the original paper, the dataset was used for a classification problem -- predicting whether or not a client has subscribed to a term deposit based on features including bank client data, data about the marketing campaigns, and some socioeconomic data. Here, we use only the bank client data to try and cluster bank clients into different groups. Such analysis may be important to banking institutions in designing marketing campaigns, as an understanding of different client groups may help banks to develop different marketing campaigns target towards each group of clients, which may potentially be be more effective than a universal marketing campaign for all clients.

**Description of the variables in the dataset:**

*Bank client data:*

1. age (numeric)
2. job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3. marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4. education (categorical: 'primary', 'secondary', 'tertiary', 'unknown')
5. default: has credit in default? (categorical: 'no','yes','unknown')
6. housing: has housing loan? (categorical: 'no','yes','unknown')
6. balance: bank balance
7. loan: has personal loan? (categorical: 'no','yes','unknown')

*Related with the last contact of the current campaign:*

9. contact: contact communication type (categorical: 'cellular','telephone')
9. month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10. day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11. duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.

*Other attributes:*

13. campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13. pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14. previous: number of contacts performed before this campaign and for this client (numeric)
15. poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')

*Social and economic context attributes:*

17. emp.var.rate: employment variation rate - quarterly indicator (numeric)
17. cons.price.idx: consumer price index - monthly indicator (numeric)
18. cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19. euribor3m: euribor 3 month rate - daily indicator (numeric)
20. nr.employed: number of employees - quarterly indicator (numeric)

*Output variable (desired target):*

22. y - has the client subscribed a term deposit? (binary: 'yes','no')



In [1]:
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from kmodes.kmodes import KModes
from scipy.cluster.hierarchy import linkage, cut_tree, dendrogram
from sklearn.decomposition import PCA
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

ModuleNotFoundError: No module named 'kmodes'

# Importing and cleaning data

The dataset we use is contained in the file bank-additional.csv, which is itself available in the zip file bank-additional.zip, which is available on the UCI Machine Learning Repository (https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank-additional.zip). Importing the dataset:

In [ ]:
data = pd.read_csv('/kaggle/input/bank-marketing-dataset/bank.csv')
data.head()

We keep only the variables related to bank client data:

In [ ]:
data = data.iloc[:, :8]
data.head()

Checking the number of rows and columns in the dataset:

In [ ]:
print(f'There are {data.shape[0]} rows and {data.shape[1]} columns.')

Checking if there are any duplicate rows:

In [ ]:
data.duplicated().sum()

There are 846 duplicated rows -- which is not a lot considering the number of rows in the dataset. We therefore remove the duplicate rows.

In [ ]:
data.drop_duplicates(inplace=True)

Checking for any missing values:

In [ ]:
data.isnull().sum()

Checking the data types of the features:

In [ ]:
data.dtypes

The dataset has one numeric variable, the age, and six categorical variables. Converting the data types of the categorical columns to the type 'category':

In [ ]:
cat_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan']

for col in cat_cols:
    data[col] = data[col].astype('category')

data.dtypes

Finally, checking the unique values of the categorical columns to see if they match the description of the dataset:

In [ ]:
for col in cat_cols:
    print(f'{col}:')
    print(data[col].unique())
    print('\n')

All categorical values match the dataset description. We now proceed to an exploratory analysis of the data.

# EDA

We begin by producing frequency plots for the categorical variables:

In [ ]:
sns.set_style('white')

fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(20, 10))
fig.subplots_adjust(hspace=0.7)

sns.histplot(x = data['job'], multiple="dodge", stat='density', shrink=0.8, common_norm=False, ax=axs[0,0])
axs[0,0].set_xticklabels(list(data['job'].unique()), rotation = 90)
sns.histplot(x = data['marital'], multiple="dodge", stat='density', shrink=0.8, common_norm=False, ax=axs[0,1])
axs[0,1].set_xticklabels(list(data['marital'].unique()), rotation = 90)
sns.histplot(x = data['education'], multiple="dodge", stat='density', shrink=0.8, common_norm=False, ax=axs[0,2])
axs[0,2].set_xticklabels(list(data['education'].unique()), rotation = 90)
sns.histplot(x = data['default'], multiple="dodge", stat='density', shrink=0.8, common_norm=False, ax=axs[1,0])
axs[1,0].set_xticklabels(list(data['default'].unique()), rotation = 90)
sns.histplot(x = data['housing'], multiple="dodge", stat='density', shrink=0.8, common_norm=False, ax=axs[1,1])
axs[1,1].set_xticklabels(list(data['housing'].unique()), rotation = 90)
sns.histplot(x = data['loan'], multiple="dodge", stat='density', shrink=0.8, common_norm=False, ax=axs[1,2])
axs[1,2].set_xticklabels(list(data['loan'].unique()), rotation = 90)

plt.suptitle('Frequency plots for the categorical variables')

for ax in axs.ravel():
    ax.set_ylabel('')

plt.show()

We see that most of the bank's clients are retired. Most of the clients are unmarried or divorced, have tertiary or primary education, have no credit in default or personal loans. The proportion of clients with and without housing loans is roughly similar.

Visualizing the distribution of ages of the clients:

In [ ]:
sns.histplot(data=data, x='age', bins=[10, 20, 30, 40, 50, 60, 70, 80, 90])
plt.title("Distribution of ages of the bank's clients")
plt.xticks([10, 20, 30, 40, 50, 60, 70, 80, 90])
plt.ylabel('Count')
plt.show()

Most of the bank's clients are between 30 to 40 years of age.

Visualizing the distribution of bank balances:

In [ ]:
sns.histplot(data=data, x='balance')
plt.title("Distribution of bank balances")
plt.ylabel('Count')
plt.show()

The bank balances are positively skewed, with most clients having less than 10,000 euros and some having a lot more.

# Clustering

#### K-Means clustering

We begin by using K-Means clustering. The K-Means clustering algorithm works by initially randomly assigning K cluster centers in p dimensional space, where p is the number of features. Then, the distances from each observation to each cluster center are calculated, and the observation is assigned to its closest cluster. Usually, the Euclidean distance is taken as the metric. Then, the cluster centers are recomputed as the mean of the observations in each cluster. This process is repeated until the cluster assignments do not change in successive iterations. The final cluster assignment depends on the initial random assignment of cluster means. Therefore, the K-Means algorithm is usually repeated several times with different cluster assignments, and the best cluster assignment, defined as the one where the sum of squared distances of datapoints to their closest cluster center is the lowest (this metric is also known as inertia). 

Here, we let the K-Means algorithm run for a maximum of 500 iterations, and repeat with 10 inital random initializations of cluster centers. We try the K-Means algorithm with 2 to 8 clusters. We then plot the inertias for the K-Means algorithms with different numbers of clusters.

Before applying the K-Means algorithm, we first one-hot encode the categorical features, and scale all the features so that they lie in the range $[0, 1]$. Doing this is important as the K-Means algorithm relies on notions of distance and hence is highly sensitive to the scale of the features.

In [ ]:
data_kmeans = pd.get_dummies(data, drop_first=True)
scaler = MinMaxScaler()
data_kmeans = scaler.fit_transform(data_kmeans)

In [ ]:
# One-hot encoding and scaling the features:

data_kmeans = pd.get_dummies(data, drop_first=True)
scaler = MinMaxScaler()
data_kmeans = scaler.fit_transform(data_kmeans)

# Running the K-Means algorithm for different numbers of clusters:

n_clusters = list(range(2, 9))
inertias = []

for n in n_clusters:
    kmeans = KMeans(n_clusters=n, init='random', n_init=10, max_iter=500, random_state=42)
    kmeans.fit(data_kmeans)
    inertia = kmeans.inertia_
    inertias.append(inertia)

# Plotting the inertia:

plt.figure(figsize=(7, 5))
plt.plot(n_clusters, inertias)
plt.axvline(3, linestyle='--', c='r')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.title('Inertias for K-Means with different numbers of clusters')
plt.show()


We can choose the optimal number of clusters using the so-called elbow method, where we look for an 'elbow' in the inertia plot, i.e., a point after which the drop in inertia becomes abruptly becomes more gradual. Hence, by the elbow method, 3 clusters would be chosen.

In [ ]:
kmeans = KMeans(n_clusters=3, init='random', n_init=10, max_iter=500, random_state=42)
data['cluster_k-means'] = kmeans.fit_predict(data_kmeans)

#### K-Modes clustering

Since our data is mostly categorical, the K-Means algorithm, which calculates the Euclidean distance from the cluster centers to each datapoint, is likely to not produce a good clustering, since the Euclidean distance between cluster centers and categorical features, which have been one-hot encoded and therefore take either 0 or 1 as values, does not make much sense. The K-Modes algorithm is a modification of K-Means more suited for categorical data. Instead of calculating the Euclidean distance between the datapoints and cluster centers, in K-Modes, a dissimilarity measure is used, which is defined as the number of features of the datapoint whose values do not match those of the cluster center. Also, the cluster centers are updated using the mode of the datapoints assigned to each cluster, instead of the mean. 

Below, we turn age and balance into categorical variables by binning them, and one-hot encode all the categorical variables. We then implement the K-Modes algorithm for 2 to 8 clusters. We then produce plots of the cost for each number of clusters. The cost of the K-Modes algorithm is defined as the sum of the dissimilarities of each datapoint.

In [ ]:
# Binning the age columns and one-hot encoding:

data_kmodes = data.drop('cluster_k-means', axis=1).copy()
data_kmodes['age_binned'] = pd.cut(data_kmodes['age'], bins=[10, 20, 30, 40, 50, 60, 70, 80, 90, 100]).astype('category')
data_kmodes.drop('age', axis=1, inplace=True)
data_kmodes['balance_binned'] = pd.cut(data_kmodes['balance'], bins=[2000, 5000, 10000, 20000, 50000, 100000]).astype('category')
data_kmodes.drop('balance', axis=1, inplace=True)
data_kmodes = pd.get_dummies(data_kmodes, drop_first=True)

# Running the K-Modes algorithm for different numbers of clusters:

n_clusters = list(range(2, 9))
costs = []

for n in n_clusters:
    kmodes = KModes(n_clusters=n, init='random', n_init=10, max_iter=500, random_state=42)
    kmodes.fit(data_kmodes)
    cost = kmodes.cost_
    costs.append(cost)

# Plotting the cost:

plt.figure(figsize=(7,5))
plt.plot(n_clusters, costs)
plt.axvline(3, linestyle='--', c='r')
plt.xlabel('Number of clusters')
plt.ylabel('Cost')
plt.title('Costs for K-Modes with different numbers of clusters')
plt.show()

The elbow method used on the cost curve suggests that the optimal number of clusters is 3. We choose 3 clusters for our final K-Modes model:

In [ ]:
kmodes = KModes(n_clusters=3, init='random', n_init=10, max_iter=500, random_state=42)
data['cluster_k-modes'] = kmodes.fit_predict(data_kmodes)

#### Hierarchical clustering

Hierarchical clustering is a bottom-up approach to clustering that does not require us to specify the number of clusters. It works by first treating each datapoint as its own cluster. The clusters that are most similar are then combined. This process is continued until all the observations are grouped into one cluster. To determine the most similar two clusters, a dissimilarity measure along with a method linkage have to be defined. The dissimilarity measure is usually defined as the Euclidean distance. An alternative is correlation based dissimilarity, which is one minus the correlation coefficient. The dissimilarity measure can be used to measure how 'dissimilar' two datapoints are. In order to extend this notion of dissimilarity to clusters, a method of linkage has to be defined. Generally, three methods of linkage are used -- complete, single and average. In all of these methods of linkage, first, the dissimilarity measures between all the datapoints in two clusters are computed. Then, in complete linkage, the maximum value of these dissimilarities between datapoints is used to determine the dissimilarity between the two clusters. In single linkage, the minimum value of the dissimilarities between the datapoints is used, and in average linkage, the mean value is used.

Hierarchical clustering can be used to prodce a dendrogram, which is a tree-like diagram that has all the datapoints as individual clusters at the bottom. As we move up the dendrogram, clusters are formed, one by one, until we end up with a single cluster containing all the datapoints at the top.

Below, we perform hierarchical clustering and plot the dendrograms using the three types of linkage previously mentioned. Euclidean distance is taken as the dissimilarity measure. As the hierarchical clustering algorithm, like the K-Means algorithm, relies on notions of Euclidean distance, the data for the hierarchical clustering model is preprocessed in the same way as for the K-Means algorithm.

In [ ]:
data_hc = data_kmeans.copy()

plt.figure(figsize=(14, 6))
hc_complete = linkage(data_hc, method='complete')
dendrogram_complete = dendrogram(hc_complete)
plt.title('Dendrogram - complete linkage')
plt.xlabel('Bank clients')
plt.ylabel('Euclidean distances')
plt.show()

plt.figure(figsize=(14, 6))
hc_average = linkage(data_hc, method='average')
dendrogram_average = dendrogram(hc_average)
plt.title('Dendrogram - average linkage')
plt.xlabel('Bank clients')
plt.ylabel('Euclidean distances')
plt.show()

plt.figure(figsize=(14, 6))
hc_single = linkage(data_hc, method='single')
dendrogram_single = dendrogram(hc_single)
plt.title('Dendrogram - single linkage')
plt.xlabel('Bank clients')
plt.ylabel('Euclidean distances')
plt.show()

It looks like using hierarchical clustering with complete linkage produces the most balanced clusters. We therefore continue our analysis using hierarchical clustering with complete linkage. We use 3 clusters:

In [ ]:
data['cluster_hc'] = cut_tree(hc_complete, 3)[:, 0]

# Dimensionality reduction using PCA and evaluation of the considered clustering algorithms

We now use principal component analysis (PCA) to obtain a 2-dimensional representation of the features. PCA is an unsupervised learning algorithm that expresses the features of a dataset as principal components, which are linear combinations of the features such that all the principal components are uncorrelated with each other, and successive principal components have diminishing variance. PCA is a useful technique to decorrelate highly correlated features, to reduce the dimensionality of datasets with very large numbers of features, or to reduce the dimensionality in order to visualize the features in two or three dimensional space. 

Here, we use PCA to find the first two principal components, which can then be plotted using a scatterplot. We then color these scatterplots by cluster for each of the clustering algorithms considered, and inspect them visually to determine the optimal clustering algorithm for our problem.

An important assumption of PCA is that all the features have the same means and standard deviations. Therefore, we first one-hot encode the original dataset containing the features, and then scale it so that each feature has zero mean and unit standard deviation, before performing PCA and printing the proportion of variance of the original features explained by the first two principal components:

In [ ]:
# Scaling the original features:

features = data.iloc[:, :8]
features = pd.get_dummies(features, drop_first=True)
scaler = StandardScaler()
features = scaler.fit_transform(features)

# Getting the first two principal components:

pca = PCA(2)
data_pca = pd.DataFrame(pca.fit_transform(features), columns=['Principal component 1', 'Principal component 2'])

# Printing the explained variance ratio:

explained_variance_ratio = np.sum(pca.explained_variance_ratio_)
print(f'Proportion of variance of the original features explained by the first two principal components: {explained_variance_ratio}')

# Appending the cluster labels to the PCA dataframe:

data_pca = pd.concat([data_pca, data.iloc[:, 8:11]], axis=1)

The first two principal components explain only around 23% of the variance of the original features. Nevertheless, we use them to plot a scatterplot and visualize the clusters formed by the different clustering algorithms:

In [ ]:
fig, axs = plt.subplots(ncols=3, figsize=(20, 5))
sns.scatterplot(data=data_pca, x='Principal component 1', y='Principal component 2', hue='cluster_k-means', ax=axs[0], palette='Set1')
axs[0].set_title('Clusters: K-Means')
sns.scatterplot(data=data_pca, x='Principal component 1', y='Principal component 2', hue='cluster_k-modes', ax=axs[1], palette='Set1')
axs[1].set_title('Clusters: K-Modes')
sns.scatterplot(data=data_pca, x='Principal component 1', y='Principal component 2', hue='cluster_hc', ax=axs[2], palette='Set1')
axs[2].set_title('Clusters: Hierarchical clustering')
fig.suptitle('Scatterplots of the first two principal components')
plt.show()

None of the algorithms considered produced well-seperated clusters on the first two principal components. It must be noted, however, that analysis based on the first two principal components in this case may not be appropriate because of two reasons. Firstly, as previously explained, the first two principal components explain only a small proportion of the total variance of the original features. Secondly, PCA is usually applied to numeric data, while we have mostly categorical features. Alternative dimensionality reduction methods, such as FAMD (factorical analysis of mixed data) may be used to get a more accurate lower dimensional representation of the data.

Since analysis based on the first two principal components does not yield any conclusive results about the relative performance of the clustering algorithms chosen, we choose the clustering algorithm that is best suited for (mostly) categorical data, i.e., the K-Modes algorithm, and use it for further analysis of our data.

In [ ]:
data.drop(['cluster_k-means', 'cluster_hc'], axis=1, inplace=True)

# Exploring the data by cluster

Plotting frequency plots for the variable 'job', by cluster:

In [ ]:
g = sns.displot(data=data, x='job', col='cluster_k-modes', multiple="dodge", stat='density', shrink=0.8, common_norm=False)
g.set_xticklabels(rotation=90)
g.fig.suptitle("Frequency plots for the variable 'job', by cluster", y=1.05)
plt.show()

We see that in clusters 0, most of the clients are blue-collar workers, followed by technicians. In cluster 1, most of the clients are technicians, with administrative and blue-collar jobs running second. In cluster 2, the overwhelming majority of clients are employed in management roles.

Plotting frequency plots for the variable 'marital', by cluster:

In [ ]:
g = sns.displot(data=data, x='marital', col='cluster_k-modes', multiple="dodge", stat='density', shrink=0.8, common_norm=False)
g.fig.suptitle("Frequency plots for the variable 'marital', by cluster", y=1.05)
plt.show()

In clsuter 0, most of the clients are married and a small proportion is divorced. In cluster 1, most of the clients are single and a small proportion is divorced. In cluster 2, most of the clients are married, some are single and a small proportion is divorced.

Plotting frequency plots for the variable 'education', by cluster:

In [ ]:
g = sns.displot(data=data, x='education', col='cluster_k-modes', multiple="dodge", stat='density', shrink=0.8, common_norm=False)
g.set_xticklabels(rotation=90)
g.fig.suptitle("Frequency plots for the variable 'education', by cluster", y=1.05)
plt.show()

In clusters 0 and 1, most of the clients have education upto the secondary level. In cluster 0, primary education is the second most common level of education, while in cluster 1, tertiary education is the second most common level of education. In cluster 2, almost all the clients have a tertiary level of education.

Plotting frequency plots for the variable 'default', by cluster:

In [ ]:
g = sns.displot(data=data, x='default', col='cluster_k-modes', multiple="dodge", stat='density', shrink=0.8, common_norm=False)
g.fig.suptitle("Frequency plots for the variable 'default', by cluster", y=1.05)
plt.show()

There are no significant differences across the different clusters in terms of whether the clients have credit in default -- in all three clusters, most clients do not have credit in default.

Plotting frequency plots for the variable 'housing', by cluster:

In [ ]:
g = sns.displot(data=data, x='housing', col='cluster_k-modes', multiple="dodge", stat='density', shrink=0.8, common_norm=False)
g.fig.suptitle("Frequency plots for the variable 'housing', by cluster", y=1.05)
plt.show()

In cluster 0, most of the clients have housing loans, while in clusters 1 and 2, most of them do not.

Plotting frequency plots for the variable 'loan', by cluster:

In [ ]:
g = sns.displot(data=data, x='loan', col='cluster_k-modes', multiple="dodge", stat='density', shrink=0.8, common_norm=False)
g.fig.suptitle("Frequency plots for the variable 'loan', by cluster", y=1.05)
plt.show()

Again, there are no significant differences across the different clusters in terms of whether the clients have housing loans -- in all three clusters, most of the clients have no personal loans.

Plotting histograms for the variable 'age', by cluster:

In [ ]:
g = sns.displot(data=data, x='age', col='cluster_k-modes', bins=[10, 20, 30, 40, 50, 60, 70, 80, 90])
g.fig.suptitle("Histograms for the variable 'age', by cluster", y=1.05)
plt.show()

In all three clusters, most of the clients are between 30 and 40 years of age. There is a larger proportion of poeple younger than 30 in cluster 1 compared to clusters 0 and 2.

Plotting histograms for the variable 'balance', by cluster:

In [ ]:
g = sns.displot(data=data, x='balance', col='cluster_k-modes', bins=[2000, 5000, 10000, 20000, 50000, 100000])
g.fig.suptitle("Histograms for the variable 'age', by cluster", y=1.05)
plt.show()

The distribution of bank balances is very similar across the clusters.

# Conclusion

Summarizing the differences between the clusters:
1. In cluster 0, most clients are blue-collar workers or technicians, are married, are educated upto the primary or secondary levels, have housing loans, and are between 30 and 60 years of age.
2. In cluster 1, most clients are technicians, blue-collar workers or work administrative jobs, are single, are educated upto the secondary or tertiary levels, do not have housing loans, and are between 20 and 40 years of age.
3. In cluster 2, most clients work management jobs, are married, are educated upto the tertiary level, do not have housing loans, and are between 30 and 60 years of age.

It may be concluded that most of the clients in cluster 2 have the highest income, based on their jobs, education, and age. Clients in clusters 0 and 1 have lower incomes, but for different reasons -- the clients in cluster 1 are in general more educated, are employed in higher paying occupations, but are younger and hence have less working experience compared to the clients in cluster 0. It may also be concluded that based on the differences in education and employment between the clusters, most of the clients in clusters 1 and 2 have a greater degree of financial literacy than those in cluster 0. 

Subsequently, it is likely that the highest proportion of clients that are certain about their need to have a term deposit and have a term deposit (or vice-versa) lie in cluster 2, followed by cluster 1 and then cluster 0. Based on the results of our clustering analysis, the bank should focus more on targeting clients in clusters 0 and 1. This could make the marketing campaign more effective as well as reduce costs of the marketing campaign.